# Fig. 6

## Fig. 6a

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
from matplotlib import rcParams
sc.set_figure_params(dpi=300, color_map='viridis')
sc.settings.verbosity = 2

In [ ]:
%pylab inline
import warnings
warnings.filterwarnings("ignore")
from glob import iglob
from tqdm import tqdm

In [ ]:
%matplotlib inline

In [ ]:
fig6a= sc.read("Fig6.h5")

In [ ]:
figsize(4,4)
sc.pl.pca(mg,color=["region"],frameon=False,save="Fig6a.pdf")

# Fig. 6b-d

In [ ]:
%%R
Sys.setenv(LANGUAGE = "en")
options(stringsAsFactors = FALSE)
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
deg<-read.csv("Fig6b_DEG.csv",sep=",",header=T)

########Fig. 6b
library(EnhancedVolcano) 
keyvals <- ifelse(deg$avg_log2FC < -0.2, '#74CAFF',
    ifelse(deg$avg_log2FC > 0.2, '#B3128A',
        'gray'))
keyvals[is.na(keyvals)] <- 'gray'
names(keyvals)[keyvals == '#B3128A'] <- 'Prefrontal cortex'
names(keyvals)[keyvals == 'gray'] <- 'mid'
names(keyvals)[keyvals == '#74CAFF'] <- 'Hippocampus'

p <- EnhancedVolcano(deg,
    lab = rownames(deg),
    x = 'avg_log2FC',
    y = 'p_val_adj', 
    xlab = bquote(~Log[2]~ 'fold change'),
    pCutoff = 10e-2,
    FCcutoff = 0.2,
    pointSize = 3,
    labSize = 8,
    colAlpha = 4/5,
    colCustom = keyvals,
    selectLab = rownames(deg)[which(names(keyvals) %in% c('Prefrontal cortex', 'Hippocampus'))]) + ggplot2::coord_cartesian(xlim=c(-1.5, 1.5)) + theme_classic(base_size = 30) + theme(legend.position="bottom")
p
ggsave("volcano_PFC_vs_HIPPO.pdf", plot=p, width=10, height=12)

########Fig. 6c
suppressPackageStartupMessages(library(clusterProfiler))
library(tidyverse)
library(KEGG.db) 
library(ggplot2)
library(RColorBrewer)
library(gridExtra)
library(enrichplot)
library(plyr)
library(ggrepel)
library(org.Hs.eg.db)
library(pheatmap)
library(RColorBrewer)
deg$gene_name<-rownames(deg)
deg$ENTREZID <- mapIds(org.Hs.eg.db,
                     keys=deg$gene_name,
                     column="ENTREZID",
                     keytype="SYMBOL",
                     multiVals="first")

deg <- deg %>% 
  dplyr::select(gene_name,ENTREZID,avg_log2FC,group) %>% 
  filter(ENTREZID!="NA") %>% 
  distinct(ENTREZID,.keep_all = T)
PFC_deg<-deg[deg$group == "Prefrontal cortex", ]
HIP_deg<-deg[deg$group == "Hippocampus", ]
kk <- enrichKEGG(gene = HIP_deg$ENTREZID,   organism = 'hsa',pvalueCutoff = 0.05)
kk1 <- setReadable(kk, 'org.Hs.eg.db',  'ENTREZID')
kk2<-as.data.frame(kk1)
View(kk2)
write.csv(kk2,file="kegg-HIPPO.csv")
GO_results <- enrichGO(gene = HIP_deg$gene_name, OrgDb = "org.Hs.eg.db", keyType = "SYMBOL", ont = "BP")
View(GO_results)
bp <- as.data.frame(GO_results)
View(bp)
write.csv(bp,file="BP-HIPPO.csv")
MF_results <- enrichGO(gene = HIP_deg$gene_name, OrgDb = "org.Hs.eg.db", keyType = "SYMBOL", ont = "MF")
MF <- as.data.frame(MF_results)
write.csv(MF,file="MF-HIPPO.csv")
CC_results <- enrichGO(gene = HIP_deg$gene_name, OrgDb = "org.Hs.eg.db", keyType = "SYMBOL", ont = "CC")
CC <- as.data.frame(CC_results)
write.csv(CC,file="CC-HIPPO.csv")
kk <- enrichKEGG(gene = PFC_deg$ENTREZID,   organism     = 'hsa',pvalueCutoff = 0.05)
kk1 <- setReadable(kk, 'org.Hs.eg.db',  'ENTREZID')
kk2<-as.data.frame(kk1)
View(kk2)
write.csv(kk2,file="kegg-PFC.csv")
GO_results <- enrichGO(gene = PFC_deg$gene_name, OrgDb = "org.Hs.eg.db", keyType = "SYMBOL", ont = "BP")
View(GO_results)
bp <- as.data.frame(GO_results)
View(bp)
write.csv(bp,file="BP-PFC.csv")
MF_results <- enrichGO(gene = PFC_deg$gene_name, OrgDb = "org.Hs.eg.db", keyType = "SYMBOL", ont = "MF")
MF <- as.data.frame(MF_results)
write.csv(MF,file="MF-PFC.csv")
CC_results <- enrichGO(gene = PFC_deg$gene_name, OrgDb = "org.Hs.eg.db", keyType = "SYMBOL", ont = "CC")
CC <- as.data.frame(CC_results)
write.csv(CC,file="CC-PFC.csv")

########Fig. 6c
mytheme = theme(axis.text.x = element_text(hjust = 0.5,size = 20), 
                axis.ticks.y = element_blank(),
                axis.text.y = element_text(size = 20), 
                axis.title.x = element_text(size = 20), 
                axis.title.y = element_text(size = 20), 
                axis.line = element_line(size = 1),
                plot.margin = unit(c(1,1,1,1), "cm"),
                plot.title = element_text(hjust = 0.5,size =  22),
                legend.title = element_text(size = 22), 
                legend.text = element_text(size = 22), 
                legend.position = "right",
                legend.background = element_rect(fill = 'transparent'))

library(readxl)
##pfc_GO_select.xlsx and hip_GO_select.xlsx=choose pathway from Related to Fig. 6c of Supplementary Table 5.xlsx
pfc = as.data.frame(read_excel("pfc_GO_select.xlsx"),sheet = 1);head(pfc)
hip = as.data.frame(read_excel("hip_GO_select.xlsx"),sheet = 1);head(hip)
kegg_down_dt <- as.data.frame(hip)
kegg_up_dt <- as.data.frame(pfc)
kegg_down_dt <- kegg_down_dt[order(kegg_down_dt$pvalue),]
kegg_up_dt <- kegg_up_dt[order(kegg_up_dt$pvalue),]
hip2 <- head(kegg_down_dt,8)
hip2$group = -1
pfc2 <- head(kegg_up_dt,8)
pfc2$group = 1
dat <- rbind(pfc2, hip2)
dat$pvalue <- -log10(dat$pvalue)
dat$pvalue <- dat$pvalue * dat$group
dat <- dat[order(dat$pvalue, decreasing = F),];
dat$threshold <- ifelse(dat$pvalue > 0,'Prefrontal cortex','Hippocampus')
dat$Description <- factor(dat$Description,levels = dat$Description) 


p4 <- ggplot(data = dat,aes(x = Description,y = pvalue,fill = threshold)) +
      geom_col() +
      #geom_hline(yintercept = c(-2,2),color = 'grey',size = 2,lty = 'dashed') + 
      coord_flip() +
      #geom_text(aes(label = GeneRatio),size =6) + 
      scale_fill_manual(values = c('Prefrontal cortex'= '#A71981','Hippocampus'='#77C1EC')) +
      labs(x = NULL, y = bquote("-"~Log[10]~"(P value)"), title = "GO Term Enrichment") 
      
     up_pathway <- length(which(dat$pvalue > 0))
     down_pathway <- length(which(dat$pvalue < 0))
     high <- nrow(dat)
p4 + geom_text(data = dat[1:up_pathway,],aes(x = Description,y = 0.1,label = Description),
               hjust = 0,color = 'black',size = 6) +
     geom_text(data = dat[(down_pathway +1):high,],aes(x = Description,y = -0.1,label = Description),
               hjust = 1,color = 'black',size = 6) +
     scale_x_discrete(labels = NULL) +

     scale_y_continuous(expand = expansion(add = c(0.1, 0.1)),
                        limits = c(-17,15), breaks = seq(-15,15,5), 
                        labels = c(15,10,5,0,5,10,15)) +
     theme_bw() + mytheme + 
     theme(legend.position = "none",
           axis.line.y = element_blank(),
           panel.grid.major = element_blank(),
           panel.grid.minor = element_blank(),
           panel.border = element_blank())

########Fig. 6d
##KEGG_select.xlsx =choose pathway from Related to Fig. 6d of Supplementary Table 5.xlsx
rm(list=ls())
gc()
kegg = as.data.frame(read_excel("KEGG_select.xlsx"),sheet = 1);head(kegg)
given_values <- c("hsa04724", "hsa04514", "hsa04728", "hsa04723", "hsa04024", "hsa04145", "hsa04142", "hsa04612", "hsa04666", "hsa04062")
kegg_sec <- subset(kegg, ID %in% given_values)
kegg_sec$neg_log10_p <- -log10(kegg_sec$pvalue)
kegg_sec <- kegg_sec[, c("ID","Group", "neg_log10_p")]
kegg_sec$neg_log10_p <- as.numeric(as.character(kegg_sec$neg_log10_p))
kegg_sec <- kegg_sec[!is.na(kegg_sec$neg_log10_p), ]
kegg_sec$HIPPO<-kegg_sec$neg_log10_p
kegg_sec$HIPPO[kegg_sec$Group == "PFC"] <- NA
print(kegg_sec$HIPPO)
kegg_sec$PFC<-kegg_sec$neg_log10_p
##hsa04514	Cell adhesion molecules also in PFC
kegg_sec$PFC[kegg_sec$Group == "HIPPO" & kegg_sec$ID != "hsa04514"] <- NA
kegg_sec$PFC[kegg_sec$ID == "hsa04514"] <- 0.000602117801055997
kegg_sec<-kegg_sec[, c("ID", "PFC","HIPPO")]
rownames(kegg_sec)<-kegg_sec$ID
kegg_sec<-kegg_sec[, c("HIPPO","PFC")]
colnames(kegg_sec)<-c("Hippocampus","Prefrontal cortex")
dat=log2(kegg_sec+1)
#group_list <- as.factor(c("Prefrontal cortex","Hippocampus"))
#table(group_list)
#n=t(scale(t(dat))) 
#n <- na.omit(dat)  
n=dat
n[n>3]=3
n[n< 1]= 1
##Fig. 6d
pheatmap(n,
  scale = "none",
  color = colorRampPalette(c("#3d4c9e", "white", "#eee814"))(100),
  show_rownames = TRUE,
  show_colnames = TRUE,
  cellwidth = 35,
  cellheight = 35,
  cluster_row = FALSE,
  cluster_col = FALSE,
border_color = "white",
na_col = "#808080")

## Fig. 6e

In [ ]:
all=sc.read("Fig6a.h5")

In [ ]:
joint_gene_bp=["PCDH9", "CADM2", "MBP","SORT1", "UNC5C", "NRXN3", "NCAM1", "DIP2B","AUTS2","PTK2","CDH20", "AKAP6", "PEX5L", "PDE3B"]

In [ ]:
sc.pl.matrixplot(all,joint_gene_bp, groupby='region',save='joint_gene_bp.pdf',vmax=1)

In [ ]:
pfc_go_gene=["PTPRC", "ATM","C3","CD74","CSF1R", "DOCK2", "JAK2", "IFNGR1", "SNCA", "TLR2", "PLCG2", "SYK"]

In [ ]:
ax = sc.pl.stacked_violin(all, pfc_go_gene,cmap='Greens',groupby='region',vmin=0,vmax=2, swap_axes=False,save="pfc_spe_gene_.bp.pdf")

In [ ]:
hip_spe_gene_bp=["NRXN1", "NRG3", "ROBO2", "HOMER1",  "NEGR1",  "PPFIA2",  "LRRC4C",  "GRID1", "DSCAM",   "KLF7",  "DSCAML1", "ROBO1", "GRIK2", "GRIA2", "CADM1",  "PDE4D"]

In [ ]:
ax = sc.pl.stacked_violin(all, hip_spe_gene_bp, cmap='Reds',groupby='region',vmin=0,vmax=2, swap_axes=False,save="hip_spe_gene_.bp.pdf")

## Fig. 6f

In [ ]:
%%R
Sys.setenv(LANGUAGE = "en")
options(stringsAsFactors = FALSE)
library(clusterProfiler)
library(org.Hs.eg.db)
library(GseaVis)
# load diffgene
deg<-read.csv("Fig6b_DEG.csv",sep=",",header=T,row.names = 1)
deg$ENTREZID <- mapIds(org.Hs.eg.db,
                       keys=rownames(deg),
                       column="ENTREZID",
                       keytype="SYMBOL",
                       multiVals="first")
deg$gene_name=rownames(deg)
deg_hip <- subset(deg, group == "Hippocampus")
deg_hip<-na.omit(deg_hip)
library(dplyr)
deg_hip$avg_log2FC <- abs(deg_hip$avg_log2FC)
gene_df <- deg_hip %>% 
  dplyr::select(avg_log2FC,ENTREZID,gene_name) %>% 
  filter(ENTREZID!="NA") %>% 
  distinct(ENTREZID,.keep_all = T)
geneList <- gene_df$avg_log2FC
names(geneList) = gene_df$ENTREZID
geneList = sort(geneList, decreasing = TRUE)
head(geneList)
gsym.fc.id.sorted <- gene_df[order(gene_df$avg_log2FC, decreasing = T),]
id.fc <- gsym.fc.id.sorted$avg_log2FC
names(id.fc) <- gsym.fc.id.sorted$ENTREZID
kk <- gseKEGG(id.fc, organism = "hsa", pvalueCutoff = 1)
dim(kk)
kk1 <- setReadable(kk, 'org.Hs.eg.db',  'ENTREZID')
kk2<-as.data.frame(kk1)
View(kk2)
write.csv(kk2,file="HIP_mg(pcdh9hi)_gsea_output.csv")
#head(kk)

kk.gsym <- setReadable(kk, 'org.Hs.eg.db', 
                    'ENTREZID')

sortkk <- kk.gsym[order(kk.gsym$enrichmentScore, decreasing = T),]
#head(sortkk)
#tail(sortkk)
#hsa04724  Glutamatergic synapse
#GRIK2/DLGAP1/HOMER1/GRIA2/GRM5/GRIA1/PLCB1/GRM7/KCNJ3/GRIN2B/GRIA4/GRIN2A
#hsa04514  Cell adhesion molecules
#hsa04728  Dopaminergic synapse
#hsa04723  Retrograde endocannabinoid signaling
#hsa04024  cAMP signaling pathway

# choose pathways
geneSetID <- c("hsa04724", "hsa04514","hsa04728","hsa04723","hsa04024")

# highlight gene names
selectedGeneID <- c("GRIK2","GRIA2","PLCB1","MAPK10","GABRB2","PDE4D","RYR2")

# define colors
mycol <- c("#088247","#58CDD9","#7A142C","#5D90BA","#431A3D","#91612D","#6E568C","#E0367A","#D8D155","#64495D","#7CC767")


for (i in sortkk$geneSetID) {
  gseaplot(kk, i)
  myGeneList <- enrichplot:::gsInfo(kk, i)
  row.names(myGeneList) <- gsym.fc$gsym
  myGeneList$id <- gsym.fc$ENTREZID 
  write.csv(myGeneList, paste0("gsea_genelist_", i, "_hip.csv"))
}

x <- kk
geneList <- position <- NULL ## to satisfy codetool

#merge multiple pathways
gsdata <- do.call(rbind, lapply(geneSetID, enrichplot:::gsInfo, object = x))
gsdata$gsym <- rep(gsym.fc.id.sorted$gene_name,5)

# running score
p.res <- ggplot(gsdata, aes_(x = ~x)) + xlab(NULL) +
  geom_line(aes_(y = ~runningScore, color= ~Description), size=1) +
  scale_color_manual(values = mycol) +
  
  #scale_x_continuous(expand=c(0,0)) + 
  geom_hline(yintercept = 0, lty = "longdash", lwd = 0.2) + 
  ylab("Enrichment\n Score") +
  
  theme_bw() +
  theme(panel.grid = element_blank()) + 
  
  theme(legend.position = "top", legend.title = element_blank(),
        legend.background = element_rect(fill = "transparent")) +
  
  theme(axis.text.y=element_text(size = 12, face = "bold"),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.line.x=element_blank(),
        axis.line.y = element_line(),
        plot.margin=margin(t=.2, r = .2, b=0, l=.2, unit="cm"))
#p.res


# rank
rel_heights <- c(1.5, .5, 1.5) 

i <- 0
for (term in unique(gsdata$Description)) {
  idx <- which(gsdata$ymin != 0 & gsdata$Description == term)
  gsdata[idx, "ymin"] <- i
  gsdata[idx, "ymax"] <- i + 1
  i <- i + 1
}
#head(gsdata)

p2 <- ggplot(gsdata, aes_(x = ~x)) +
  geom_linerange(aes_(ymin=~ymin, ymax=~ymax, color=~Description)) +
  xlab(NULL) + ylab(NULL) + 
  scale_color_manual(values = mycol) + 
  theme_bw() +
  theme(panel.grid = element_blank()) + 

  theme(legend.position = "none",
        plot.margin = margin(t=-.1, b=0,unit="cm"),
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        axis.line.y = element_line(),
        axis.line.x = element_blank()) +
  #scale_x_continuous(expand=c(0,0)) +
  scale_y_continuous(expand=c(0,0))
#p2


# fold changes
df2 <- p.res$data
df2$y <- p.res$data$geneList[df2$x]
df2$gsym <- p.res$data$gsym[df2$x]

#head(df2)
selectgenes <- data.frame(gsym = selectedGeneID)
selectgenes <- merge(selectgenes, df2, by = "gsym")
selectgenes <- selectgenes[selectgenes$position == 1,]
head(selectgenes)


p.pos <- ggplot(selectgenes, aes(x, y, fill = Description, color = Description, label = gsym)) + 
  geom_segment(data=df2, aes_(x=~x, xend=~x, y=~y, yend=0), 
               color = "grey") +
  geom_bar(position = "dodge", stat = "identity") +
  scale_fill_manual(values = mycol, guide=FALSE) + 
  scale_color_manual(values = mycol, guide=FALSE) + 

  #scale_x_continuous(expand=c(0,0)) +
  geom_hline(yintercept = 0, lty = 2, lwd = 0.2) +
  ylab("Ranked list\n metric") +
  xlab("Rank in ordered dataset") +
  
  theme_bw() +
  theme(axis.text.y=element_text(size = 12, face = "bold"),axis.line.y = element_line(),
        panel.grid = element_blank()) +

  # show gene names
  geom_text_repel(data = selectgenes, 
                  show.legend = FALSE, 
                  direction = "x", 
                  ylim = c(2, NA), 
                  angle = 90, 
                  size = 2.5, box.padding = unit(0.35, "lines"), 
        point.padding = unit(0.3, "lines")) +
  theme(plot.margin=margin(t = -.1, r = .2, b=.2, l=.2, unit="cm"))
#p.pos

library("cowplot")
# 组图
plotlist <- list(p.res, p2, p.pos)
n <- length(plotlist)
plotlist[[n]] <- plotlist[[n]] +
  theme(axis.line.x = element_line(),
        axis.ticks.x = element_line(),
        axis.text.x = element_text(size = 12, face = "bold"))

plot_grid(plotlist = plotlist, ncol = 1, align="v", rel_heights = rel_heights)

# Fig. 6g

In [ ]:
%%R
Sys.setenv(LANGUAGE = "en")
options(stringsAsFactors = FALSE)
library(clusterProfiler)
library(org.Hs.eg.db)
library(GseaVis)
# load diffgene
deg<-read.csv("Fig6b_DEG.csv",sep=",",header=T,row.names = 1)
deg$ENTREZID <- mapIds(org.Hs.eg.db,
                       keys=rownames(deg),
                       column="ENTREZID",
                       keytype="SYMBOL",
                       multiVals="first")
deg$gene_name=rownames(deg)
deg_pfc <- subset(deg, group == "Prefrontal cortex")
deg_pfc<-na.omit(deg_pfc)
library(dplyr)
gene_df <- deg_pfc %>% 
  dplyr::select(avg_log2FC,ENTREZID,gene_name) %>% 
  filter(ENTREZID!="NA") %>% 
  distinct(ENTREZID,.keep_all = T)
geneList <- gene_df$avg_log2FC
names(geneList) = gene_df$ENTREZID
geneList = sort(geneList, decreasing = TRUE)
head(geneList)
gsym.fc.id.sorted <- gene_df[order(gene_df$avg_log2FC, decreasing = T),]
id.fc <- gsym.fc.id.sorted$avg_log2FC
names(id.fc) <- gsym.fc.id.sorted$ENTREZID
kk <- gseKEGG(id.fc, organism = "hsa", pvalueCutoff = 1)
dim(kk)
kk.gsym <- setReadable(kk, 'org.Hs.eg.db', 
                    'ENTREZID')
sortkk <- kk.gsym[order(kk.gsym$enrichmentScore, decreasing = T),]

##choose pathway
geneSetID <- c("hsa04145", "hsa04142","hsa04612","hsa04666","hsa04062")

##highlight genes
selectedGeneID <- c("CTSB","LAMP2","C3","MSR1","CD74","PTPRC","PLCG2","JAK2")

###define colors
mycol <- c("darkgreen","chocolate4","blueviolet","#223D6C","#D20A13","#088247","#58CDD9","#7A142C","#5D90BA","#431A3D","#91612D","#6E568C","#E0367A","#D8D155","#64495D","#7CC767")
for (i in sortkk$geneSetID) {
  gseaplot(kk, i)
  myGeneList <- enrichplot:::gsInfo(kk, i)
  row.names(myGeneList) <- gsym.fc$gsym
  myGeneList$id <- gsym.fc$ENTREZID 
  write.csv(myGeneList, paste0("gsea_genelist_", i, "_pfc.csv"))
}

x <- kk
geneList <- position <- NULL ## to satisfy codetool

# merge multiple pathways
gsdata <- do.call(rbind, lapply(geneSetID, enrichplot:::gsInfo, object = x))
gsdata$gsym <- rep(gsym.fc.id.sorted$gene_name,5)

# running score
p.res <- ggplot(gsdata, aes_(x = ~x)) + xlab(NULL) +
  geom_line(aes_(y = ~runningScore, color= ~Description), size=1) +
  scale_color_manual(values = mycol) +
  
  #scale_x_continuous(expand=c(0,0)) + 
  geom_hline(yintercept = 0, lty = "longdash", lwd = 0.2) + 
  ylab("Enrichment\n Score") +
  
  theme_bw() +
  theme(panel.grid = element_blank()) + 
  
  theme(legend.position = "top", legend.title = element_blank(),
        legend.background = element_rect(fill = "transparent")) +
  
  theme(axis.text.y=element_text(size = 12, face = "bold"),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.line.x=element_blank(),
        axis.line.y = element_line(),
        plot.margin=margin(t=.2, r = .2, b=0, l=.2, unit="cm"))
#p.res

# rank
rel_heights <- c(1.5, .5, 1.5) 

i <- 0
for (term in unique(gsdata$Description)) {
  idx <- which(gsdata$ymin != 0 & gsdata$Description == term)
  gsdata[idx, "ymin"] <- i
  gsdata[idx, "ymax"] <- i + 1
  i <- i + 1
}
#head(gsdata)

p2 <- ggplot(gsdata, aes_(x = ~x)) +
  geom_linerange(aes_(ymin=~ymin, ymax=~ymax, color=~Description)) +
  xlab(NULL) + ylab(NULL) + 
  scale_color_manual(values = mycol) + 

  theme_bw() +
  theme(panel.grid = element_blank()) + 

  theme(legend.position = "none",
        plot.margin = margin(t=-.1, b=0,unit="cm"),
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        axis.line.y = element_line(),
        axis.line.x = element_blank()) +
  #scale_x_continuous(expand=c(0,0)) +
  scale_y_continuous(expand=c(0,0))
#p2


# plot fold change
df2 <- p.res$data
df2$y <- p.res$data$geneList[df2$x]
df2$gsym <- p.res$data$gsym[df2$x]

#head(df2)

# extract fold change
selectgenes <- data.frame(gsym = selectedGeneID)
selectgenes <- merge(selectgenes, df2, by = "gsym")
selectgenes <- selectgenes[selectgenes$position == 1,]
head(selectgenes)


p.pos <- ggplot(selectgenes, aes(x, y, fill = Description, color = Description, label = gsym)) + 
  geom_segment(data=df2, aes_(x=~x, xend=~x, y=~y, yend=0), 
               color = "grey") +
  geom_bar(position = "dodge", stat = "identity") +
  scale_fill_manual(values = mycol, guide=FALSE) + 
  scale_color_manual(values = mycol, guide=FALSE) + 

  #scale_x_continuous(expand=c(0,0)) +
  geom_hline(yintercept = 0, lty = 2, lwd = 0.2) +
  ylab("Ranked list\n metric") +
  xlab("Rank in ordered dataset") +
  
  theme_bw() +
  theme(axis.text.y=element_text(size = 12, face = "bold"),
        panel.grid = element_blank()) +

  # show gene name
  geom_text_repel(data = selectgenes, 
                  show.legend = FALSE, 
                  direction = "x", 
                  ylim = c(2, NA), 
                  angle = 90, 
                  size = 2.5, box.padding = unit(0.35, "lines"), 
        point.padding = unit(0.3, "lines")) +
  theme(plot.margin=margin(t = -.1, r = .2, b=.2, l=.2, unit="cm"))
#p.pos

library("cowplot")
# group image
plotlist <- list(p.res, p2, p.pos)
n <- length(plotlist)
plotlist[[n]] <- plotlist[[n]] +
  theme(axis.line.x = element_line(),
        axis.ticks.x = element_line(),
        axis.line.y = element_line(),
        axis.text.x = element_text(size = 12, face = "bold"))

plot_grid(plotlist = plotlist, ncol = 1, align="v", rel_heights = rel_heights)
